In [65]:
import openai

In [66]:
from dotenv import load_dotenv

In [67]:
import os

In [68]:
openai.api_key=""

In [69]:
completion=openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role":"system","content":"You are an assistant whch informs about temperature"},
        {"role":"user","content":"Hy there"}
    ]
)

In [70]:
completion.choices[0].message

<OpenAIObject at 0x188ffec7d10> JSON: {
  "role": "assistant",
  "content": "Hello! How can I assist you today?"
}

In [71]:
import requests
import json

def get_current_weather(location):
    url = "https://ai-weather-by-meteosource.p.rapidapi.com/find_places"

    querystring = {"text":location}

    headers = {
        
	     "X-RapidAPI-Key": "fb0f7df7a4msh7e96ef2aeb9acdep101832jsn5953347c86dc",
	     "X-RapidAPI-Host": "ai-weather-by-meteosource.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)

    print(response.json())



In [74]:
response=get_current_weather('Hyderabad')

[{'name': 'Hyderabad', 'place_id': 'hyderabad', 'adm_area1': 'Telangana', 'adm_area2': 'Hyderabad', 'country': 'India', 'lat': '17.38405N', 'lon': '78.45636E', 'timezone': 'Asia/Kolkata', 'type': 'settlement'}, {'name': 'Hyderābād', 'place_id': 'hyderabad-1176734', 'adm_area1': 'Sindh', 'adm_area2': 'Hyderābād', 'country': 'Islamic Republic of Pakistan', 'lat': '25.39242N', 'lon': '68.37366E', 'timezone': 'Asia/Karachi', 'type': 'settlement'}, {'name': 'Chak Fifty-four', 'place_id': 'chak-fiftyfour-1382731', 'adm_area1': 'Punjab', 'adm_area2': 'Khānewāl', 'country': 'Islamic Republic of Pakistan', 'lat': '30.47827N', 'lon': '72.23856E', 'timezone': 'Asia/Karachi', 'type': 'settlement'}, {'name': 'Hyderabad', 'place_id': 'hyderabad-1418299', 'adm_area1': 'Punjab', 'adm_area2': 'Lodhrān', 'country': 'Islamic Republic of Pakistan', 'lat': '29.78445N', 'lon': '71.89209E', 'timezone': 'Asia/Karachi', 'type': 'settlement'}, {'name': 'Hyderabad', 'place_id': 'hyderabad-1177364', 'adm_area1': 

In [75]:
print(response)

None


In [76]:
functions = [
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type":"object",
                "properties":{
                    "locations":{
                        "type":"string",
                        "description":"The city and state , eg. San Francisco, CA"
                    }
                },
                "required":["location"]
            },
        },    
    ]

In [77]:
functions

[{'name': 'get_current_weather',
  'description': 'Get the current weather in a given location',
  'parameters': {'type': 'object',
   'properties': {'locations': {'type': 'string',
     'description': 'The city and state , eg. San Francisco, CA'}},
   'required': ['location']}}]

In [78]:
user_message="Hi"
messages=[]

messages.append({"role":"user","content":user_message})

completion=openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=messages
)

In [79]:
print(completion.choices[0].message)

{
  "role": "assistant",
  "content": "Hello! How can I assist you today?"
}


In [85]:
user_message="What is the temperature of Hyderabad"
messages=[]

messages.append({"role":"user","content":user_message})

completion=openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=messages,
    functions=functions
)

In [86]:
print(completion)

{
  "id": "chatcmpl-96dTPuvgkJ0qfF4Je9r4EnKIrEHfS",
  "object": "chat.completion",
  "created": 1711368731,
  "model": "gpt-3.5-turbo-0125",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_current_weather",
          "arguments": "{\"locations\":\"Hyderabad\"}"
        }
      },
      "logprobs": null,
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 68,
    "completion_tokens": 16,
    "total_tokens": 84
  },
  "system_fingerprint": "fp_3bc1b5746c"
}


In [87]:
print(completion.choices[0].message)

{
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_current_weather",
    "arguments": "{\"locations\":\"Hyderabad\"}"
  }
}


In [88]:
response=completion.choices[0].message

In [89]:
print(response)

{
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_current_weather",
    "arguments": "{\"locations\":\"Hyderabad\"}"
  }
}


In [90]:
function_name=response['function_call']['name']

In [91]:
response['function_call']['arguments']

'{"locations":"Hyderabad"}'

In [92]:
import json
#eval is used to convert string to jsons
location=eval(response['function_call']['arguments'])['locations']

In [93]:
location

'Hyderabad'

In [94]:
messages.append(response)
messages.append(
    {
        "role":"function",
        "name":function_name,
        "content":location,
    }
)

In [95]:
messages

[{'role': 'user', 'content': 'What is the temperature of Hyderabad'},
 <OpenAIObject at 0x188ffeaaa90> JSON: {
   "role": "assistant",
   "content": null,
   "function_call": {
     "name": "get_current_weather",
     "arguments": "{\"locations\":\"Hyderabad\"}"
   }
 },
 {'role': 'function', 'name': 'get_current_weather', 'content': 'Hyderabad'}]

In [98]:
completion=openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=messages,
    functions=functions
)

In [99]:
response=completion.choices[0].message

In [101]:
print(response["content"])

The current temperature in Hyderabad is 28°C.
